In [38]:
import tensorflow

In [39]:
import keras

In [40]:
print(1)

1


In [41]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [42]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5764561415883558323
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281553818
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8576835601626111354
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [6]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [31]:
img_width, img_height = 224, 224
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [8]:
nr_categories = 102
batch_size = 16
epochs = 50

In [9]:
model = applications.nasnet.NASNetLarge(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#from keras.models import load_model
#model = load_model("/data/vgg16_fungi_all_p2.h5")

343613440/343610240 [==============================] - 3s 0us/step


In [10]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 stem_conv1
2 stem_bn1
3 activation_1
4 reduction_conv_1_stem_1
5 reduction_bn_1_stem_1
6 activation_2
7 activation_4
8 separable_conv_1_reduction_left1_stem_1
9 separable_conv_1_reduction_right1_stem_1
10 separable_conv_1_bn_reduction_left1_stem_1
11 separable_conv_1_bn_reduction_right1_stem_1
12 activation_3
13 activation_5
14 separable_conv_2_reduction_left1_stem_1
15 separable_conv_2_reduction_right1_stem_1
16 separable_conv_2_bn_reduction_left1_stem_1
17 separable_conv_2_bn_reduction_right1_stem_1
18 activation_6
19 reduction_add_1_stem_1
20 separable_conv_1_reduction_right2_stem_1
21 activation_8
22 activation_10
23 separable_conv_1_bn_reduction_right2_stem_1
24 separable_conv_1_reduction_right3_stem_1
25 separable_conv_1_reduction_left4_stem_1
26 activation_7
27 separable_conv_1_bn_reduction_right3_stem_1
28 separable_conv_1_bn_reduction_left4_stem_1
29 separable_conv_2_reduction_right2_stem_1
30 activation_9
31 activation_11
32 reduction_left2_stem_1
33 separable_con

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 96) 2592        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 96) 0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

cropping2d_4 (Cropping2D)       (None, 14, 14, 2016) 0           zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
reduction_concat_reduce_12 (Con (None, 7, 7, 2688)   0           reduction_add_2_reduce_12[0][0]  
                                                                 reduction_add3_reduce_12[0][0]   
                                                                 add_4[0][0]                      
                                                                 reduction_add4_reduce_12[0][0]   
__________________________________________________________________________________________________
adjust_avg_pool_1_13 (AveragePo (None, 7, 7, 2016)   0           adjust_relu_1_13[0][0]           
__________________________________________________________________________________________________
adjust_avg_pool_2_13 (AveragePo (None, 7, 7, 2016)   0           cropping2d_4[0][0]               
__________

In [12]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in model.layers[:]:
    print(layer.name)
    layer.trainable = False

input_1
stem_conv1
stem_bn1
activation_1
reduction_conv_1_stem_1
reduction_bn_1_stem_1
activation_2
activation_4
separable_conv_1_reduction_left1_stem_1
separable_conv_1_reduction_right1_stem_1
separable_conv_1_bn_reduction_left1_stem_1
separable_conv_1_bn_reduction_right1_stem_1
activation_3
activation_5
separable_conv_2_reduction_left1_stem_1
separable_conv_2_reduction_right1_stem_1
separable_conv_2_bn_reduction_left1_stem_1
separable_conv_2_bn_reduction_right1_stem_1
activation_6
reduction_add_1_stem_1
separable_conv_1_reduction_right2_stem_1
activation_8
activation_10
separable_conv_1_bn_reduction_right2_stem_1
separable_conv_1_reduction_right3_stem_1
separable_conv_1_reduction_left4_stem_1
activation_7
separable_conv_1_bn_reduction_right3_stem_1
separable_conv_1_bn_reduction_left4_stem_1
separable_conv_2_reduction_right2_stem_1
activation_9
activation_11
reduction_left2_stem_1
separable_conv_2_bn_reduction_right2_stem_1
separable_conv_2_reduction_right3_stem_1
separable_conv_2_red

In [17]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [18]:
model_final = Model(input = model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [19]:
model_final.shape

AttributeError: 'Model' object has no attribute 'shape'

In [20]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
# for layer in model_final.layers[:12]:
#     print(layer.name)
#     layer.trainable = False

In [21]:
# model_final.summary()

In [22]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [23]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2)

In [24]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [32]:
#img_width, img_height = 256, 256
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 4604 images belonging to 102 classes.


In [33]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 1094 images belonging to 102 classes.


In [34]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("NASNetLarge_oxford102_train_dataset.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [35]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [36]:
#batch 16

In [ ]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
288/287 [==============================] - 234s 811ms/step - loss: 3.4294 - acc: 0.2867 - val_loss: 3.6931 - val_acc: 0.2057

Epoch 00001: val_acc improved from -inf to 0.20567, saving model to inception_resnet_v2_oxford102_train_dataset.h5
Epoch 2/50
288/287 [==============================] - 218s 757ms/step - loss: 1.3560 - acc: 0.6732 - val_loss: 3.5821 - val_acc: 0.2349

Epoch 00002: val_acc improved from 0.20567 to 0.23492, saving model to inception_resnet_v2_oxford102_train_dataset.h5
Epoch 3/50
288/287 [==============================] - 218s 758ms/step - loss: 0.6789 - acc: 0.8369 - val_loss: 3.8685 - val_acc: 0.2203

Epoch 00003: val_acc did not improve from 0.23492
Epoch 4/50
288/287 [==============================] - 218s 757ms/step - loss: 0.3991 - acc: 0.9069 - val_loss: 3.5738 - val_acc: 0.2587

Epoch 00004: val_acc improved from 0.23492 to 0.25868, saving model to inception_resnet_v2_oxford102_train_dataset.h5
Epoch 5/50
 67/287 [=====>........................]

In [ ]:
#batch 256


In [ ]:
print(10)

In [ ]:
#cp inception_resnet_v2_oxford102_train_dataset.h5 NASNetLarge_oxford102_train_dataset.h5